In [178]:
import tensorflow as tf

In [179]:
import numpy as np 
import pandas as pd 
import os
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout,BatchNormalization
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

from tensorflow.keras.applications.inception_v3 import InceptionV3

In [180]:
image_size = [128,128]
data_path = './Basededatos/'

In [170]:
conv = VGG16(input_shape= image_size+[3],weights='imagenet',include_top=False)

In [171]:
conv.output

<KerasTensor: shape=(None, 4, 4, 512) dtype=float32 (created by layer 'block5_pool')>

In [172]:
for layer in conv.layers:
    layer.trainable = False

In [173]:
x = conv.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x)
x = Dense(1024,activation='relu')(x)
x = Dense(512, activation='relu')(x)
x=  Dropout(.2)(x)
pred = Dense(2,activation='softmax')(x)
model = Model(inputs = conv.input,outputs=pred)

In [174]:
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0   

In [175]:
train_datagen=ImageDataGenerator(
    rescale=1/255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    vertical_flip=True,
    validation_split=0.2)

train_generator=train_datagen.flow_from_directory('./Basededatos/', target_size=(128,128),  shuffle=False, subset='training', batch_size=10, class_mode='categorical')
val_generator = train_datagen.flow_from_directory('./Basededatos/', target_size=(128,128),  shuffle=False, subset='validation', batch_size=10, class_mode='categorical')

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

Found 471 images belonging to 2 classes.
Found 117 images belonging to 2 classes.


In [177]:
print(train_generator.n)
print(train_generator.batch_size)
print(242//32)

471
10
7


In [140]:
print(val_generator.n)
print(val_generator.batch_size)
#Next we have to create a labels.txt file that will hold all our labels (important for Flutter)
print(train_generator.class_indices) #prints every single key and class of that dataset
labels = '\n'.join(sorted(train_generator.class_indices.keys())) #print all these keys as a list of labels into a text file called labels.txt
with open('labels.txt', 'w') as f: #writes to the labels.txt file, and if it doesnt exists, it creates one, and if it does exist, it will overrite it. (thats what 'w' is for)
    f.write(labels)

#preprocessing of raw data is hence complete and now its time to build our neural network

117
10
{'Con tumor': 0, 'Sin tumor': 1}


In [141]:
step_size_train=train_generator.n//train_generator.batch_size
history = model.fit_generator(generator=train_generator, steps_per_epoch=step_size_train, epochs=60,  verbose=1, validation_data=val_generator)


Epoch 1/60
47/47 [==============================] - 13s 266ms/step - loss: 2.6875 - accuracy: 0.7744 - val_loss: 0.4807 - val_accuracy: 0.8291
Epoch 2/60
47/47 [==============================] - 13s 268ms/step - loss: 0.4725 - accuracy: 0.8286 - val_loss: 0.3739 - val_accuracy: 0.8376
Epoch 3/60
47/47 [==============================] - 12s 262ms/step - loss: 0.3159 - accuracy: 0.8677 - val_loss: 0.5129 - val_accuracy: 0.7179
Epoch 4/60
47/47 [==============================] - 12s 258ms/step - loss: 0.2817 - accuracy: 0.9046 - val_loss: 0.3641 - val_accuracy: 0.8803
Epoch 5/60
47/47 [==============================] - 13s 276ms/step - loss: 0.4219 - accuracy: 0.8547 - val_loss: 0.3126 - val_accuracy: 0.8376
Epoch 6/60
47/47 [==============================] - 12s 264ms/step - loss: 0.4459 - accuracy: 0.8742 - val_loss: 0.4877 - val_accuracy: 0.8291
Epoch 7/60
47/47 [==============================] - 12s 263ms/step - loss: 0.4348 - accuracy: 0.8612 - val_loss: 0.3684 - val_accuracy: 0.8718

In [145]:
import matplotlib.pyplot as plt
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
t = f.suptitle('VGG16', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

epoch_list = list(range(0, len(history.history['accuracy'])))
ax1.plot(epoch_list, history.history['accuracy'], label='Train Accuracy')
ax1.plot(epoch_list, history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_xticks(np.arange(0, len(history.history['accuracy']), 5))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(0, len(history.history['accuracy']), 5))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")

In [146]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [147]:
#Confution Matrix and Classification Report
Y_pred = model.predict_generator(val_generator, val_generator.n // val_generator.batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)

In [149]:
print(val_generator.classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1]


In [150]:
print('Confusion Matrix')
print(confusion_matrix(val_generator.classes, y_pred))

Confusion Matrix
[[90  7]
 [ 3 17]]


In [151]:
print('Classification Report')
target_names = ['No Tumor', 'Tumor']
print(classification_report(val_generator.classes, y_pred, target_names=target_names))

Classification Report
              precision    recall  f1-score   support

    No Tumor       0.97      0.93      0.95        97
       Tumor       0.71      0.85      0.77        20

    accuracy                           0.91       117
   macro avg       0.84      0.89      0.86       117
weighted avg       0.92      0.91      0.92       117



In [152]:
import seaborn as sns
cm=confusion_matrix(val_generator.classes, y_pred)
sns.heatmap(cm, square=True, annot=True, cbar=False, cmap=plt.cm.Blues)
plt.xlabel('Predicted Values')
plt.ylabel('True Values')

Text(700.5483091787441, 0.5, 'True Values')

In [153]:
import seaborn as sns
import matplotlib.pyplot as plt     

ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax); #annot=True to annotate cells

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['No Tumor', 'Tumor']); ax.yaxis.set_ticklabels(['Tumor', 'No Tumor'])

[Text(0, 0.5, 'Tumor'), Text(0, 1.5, 'No Tumor')]

In [154]:
from sklearn.metrics import accuracy_score
results = confusion_matrix(val_generator.classes,y_pred) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(val_generator.classes,y_pred)) 
print('Report : ')
print(classification_report(val_generator.classes,y_pred))

sns.heatmap(results/np.sum(results), annot=True, 
            fmt='.2%', cmap='Blues')

Confusion Matrix :
[[90  7]
 [ 3 17]]
Accuracy Score : 0.9145299145299145
Report : 
              precision    recall  f1-score   support

           0       0.97      0.93      0.95        97
           1       0.71      0.85      0.77        20

    accuracy                           0.91       117
   macro avg       0.84      0.89      0.86       117
weighted avg       0.92      0.91      0.92       117



<Axes: title={'center': 'Confusion Matrix'}>

In [155]:
import numpy as np
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(val_generator.classes, y_pred, pos_label=1)
metrics.auc(fpr, tpr)

0.8889175257731959

In [156]:
fig, ax = plt.subplots(figsize=(8,5))
ax.plot(fpr, tpr)
ax.plot(np.linspace(0, 1, 100),
         np.linspace(0, 1, 100),
         label='baseline',
         linestyle='--')
plt.title('Receiver Operating Characteristic Curve', fontsize=14)
plt.ylabel('Total Positive Rate', fontsize=12)
plt.xlabel('False Positive Rate', fontsize=12)
plt.legend(fontsize=12);

In [157]:
model.save('Vgg16Detection.h5')

In [158]:
from keras.models import Model, load_model
model = load_model('Vgg16Detection.h5')

In [159]:
import gradio as gr

In [160]:
def load_labels(file_path):
    with open(file_path, 'r') as f:
        labels = f.read().splitlines()
    return labels

# Especifica la ruta del archivo labels.txt
labels_file = 'labels.txt'

# Carga las clases desde el archivo labels.txt
class_names = load_labels(labels_file)

# Verifica las clases cargadas
print(class_names)

['Con tumor', 'Sin tumor']


In [161]:
def predict_image(img):
  img_4d=img.reshape(-1,128,128,3)
  prediction=model.predict(img_4d)[0]
  return {class_names[i]: float(prediction[i]) for i in range(2)}

In [162]:
import cv2
import gradio as gr
import numpy as np

def predict_image(img):
    img_resized = cv2.resize(img, (128, 128))  # Redimensionar la imagen a 128x128
    img_resized = img_resized / 255.0  # Normalizar los valores de píxeles entre 0 y 1
    img_resized = np.expand_dims(img_resized, axis=0)  # Agregar una dimensión adicional para el tamaño del lote

    # Realizar la predicción utilizando el modelo
    predictions = model.predict(img_resized)
    predicted_class = np.argmax(predictions)  # Obtener el índice de la clase predicha
    confidence = float(predictions[0, predicted_class])  # Obtener la confianza de la predicción

    label = class_names[predicted_class]  # Obtener la etiqueta correspondiente a la clase predicha

    return {label: confidence}

In [166]:
import gradio as gr

image = gr.inputs.Image(shape=(128, 128))
label = gr.outputs.Label(num_top_classes=2)

gr.Interface(fn=predict_image, inputs=image, outputs=label, interpretation='default').launch(debug=True)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 47ms/step
Keyboard interruption in main thread... closing server.
